In [1]:
import logging
import os
from pathlib import Path

import cm1.input.sounding
import pandas as pd
import xarray
from cm1.run import PBS, CM1Run
from cm1.utils import animate_cm1out_nc, skewt
from metpy.units import units

In [2]:
# Path to the CM1 repository
cm1_path = Path(os.getenv("SCRATCH")) / "cm1"

In [3]:
testcase = "squall_line"
run_dir = cm1_path / f"run_{testcase}"

pbs_config = PBS(
    name=testcase,
    account="NMMM0021",
    walltime="01:00:00",
    nodes=1,
    run_dir=run_dir,
    executable_path=cm1_path / "run/cm1.exe",
)

run_serial = testcase.startswith("scm_") or testcase == "nh_mountain_waves"
if run_serial:
    pbs_config.executable_path = cm1_path / "run/cm1.single.gnu.exe"

In [4]:
cm1_run = CM1Run(cm1_path=cm1_path, pbs_config=pbs_config)
# Modify values in namelist
namelist = cm1_run.namelist
# Combine output in one netCDF file.
namelist["param9"]["output_format"] = 2
namelist["param9"]["output_filetype"] = 1

if testcase in ["supercell"]:
    namelist["param2"]["isnd"] = 7
    ofile = run_dir / "input_sounding"
    if not os.path.exists(ofile):
        valid_time = pd.to_datetime("20240526")
        lon = -98 * units.degree_E
        lat = 34 * units.degree_N

        if os.path.exists("/glade/campaign") or os.path.exists("/campaign"):
            input_sounding_ds = cm1.input.sounding.era5_model_level(valid_time, lon=lon, lat=lat)
        else:
            input_sounding_ds = cm1.input.sounding.era5_aws(valid_time, lon=lon, lat=lat)
        with open(ofile, "w") as f:
            f.write(cm1.input.sounding.to_txt(input_sounding_ds))
            logging.warning(ofile)

if run_serial:
    cm1_run.run_serial(background=False)
else:
    cm1_run.submit_job()
print(cm1_run.readme)

FileExistsError: Output file /glade/derecho/scratch/ahijevyc/cm1/run_squall_line/cm1out.nc already exists.

In [5]:
ifile = run_dir / "input_sounding"
if os.path.exists(ifile):
    skewt(cm1.input.sounding.read_from_txt(ifile))

In [6]:
# Open NetCDF file
output_ds = xarray.open_dataset(run_dir / "cm1out.nc")

animate_cm1out_nc(output_ds, "dbz", height=0.1, dim="zh", vmin=-16, vmax=60, cmap="gist_ncar")

In [7]:
print(cm1_run.readme)


  Idealized squall line simulation, mostly following Bryan et al (2003,
  MWR, pg 2394). 

  Default configuration is the 1-km weak-shear case.




In [6]:
help(CM1Run)

Help on class CM1Run in module cm1.run:

class CM1Run(builtins.object)
 |  CM1Run(cm1_path: pathlib.Path, pbs_config: cm1.run.PBS, namelist_path: Optional[pathlib.Path] = None, printout: Optional[str] = 'cm1.print.out')
 |
 |  Methods defined here:
 |
 |  __init__(self, cm1_path: pathlib.Path, pbs_config: cm1.run.PBS, namelist_path: Optional[pathlib.Path] = None, printout: Optional[str] = 'cm1.print.out')
 |      Initialize a CM1 model run.
 |      Assign a default namelist and readme.
 |
 |      :param cm1_path: Path to CM1 repository.
 |      :param pbs_config: Instance of PBS.
 |      :param namelist_path: Path to namelist.
 |      :param printout: Filename for the standard output log.
 |
 |  generate_pbs_script(self, script_path: str = 'pbs.job') -> str
 |      Generate a PBS job script for the CM1 model run.
 |
 |      :param script_path: Name of the PBS script file.
 |      :return: Full path of the generated PBS script.
 |
 |  prepare_run_dir(self) -> None
 |      Prepare the ru